In [1]:
import pandas as pd

# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class



In [2]:
df_movies = pd.read_csv(r'c:\Users\mkskr\OneDrive\netology\dz\dz12\ml-latest-small\movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df_ratings = pd.read_csv(r'c:\Users\mkskr\OneDrive\netology\dz\dz12\ml-latest-small\ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
df_movies = df_movies.merge(df_ratings, on=['movieId'] , how='left')
df_movies = df_movies.groupby(['title', 'genres']).mean().reset_index()[['title', 'genres', 'rating']].sort_values('title', ascending=False).reset_index(drop=True)
df_movies.head()

,title,genres,rating
0,İtirazım Var (2014),Action|Crime|Drama,3.500000
1,À nous la liberté (Freedom for Us) (1931),Comedy|Musical,4.500000
2,¡Three Amigos! (1986),Comedy|Western,3.258065
3,xXx: State of the Union (2005),Action|Crime|Thriller,1.000000
4,xXx (2002),Action|Crime|Thriller,2.478261


In [5]:
def class_ratings(row):
    if row['rating'] <= 2:
        return 'низкий ретйинг'
    elif row['rating'] > 2 and row['rating'] <=4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [6]:
df_movies['class'] = df_movies.apply(class_ratings, axis=1)
df_movies.head()

,title,genres,rating,class
0,İtirazım Var (2014),Action|Crime|Drama,3.500000,средний рейтинг
1,À nous la liberté (Freedom for Us) (1931),Comedy|Musical,4.500000,высокий рейтинг
2,¡Three Amigos! (1986),Comedy|Western,3.258065,средний рейтинг
3,xXx: State of the Union (2005),Action|Crime|Thriller,1.000000,низкий ретйинг
4,xXx (2002),Action|Crime|Thriller,2.478261,средний рейтинг


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.


In [7]:
df_keywords = pd.read_csv('keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [8]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [9]:
def geo_classificator (row):
    for region, city_list in geo_data.items():
        for citi in city_list:
            if citi in row:
                return region
    else:
        return "undefind"   

In [10]:
df_keywords.loc[:, 'content'] = df_keywords['keyword'].str.split(' ')
df_keywords.head()

,keyword,shows,content
0,вк,64292779,[вк]
1,одноклассники,63810309,[одноклассники]
2,порно,41747114,[порно]
3,ютуб,39995567,[ютуб]
4,вконтакте,21014195,[вконтакте]


In [11]:
df_keywords.loc[:,"region"] = df_keywords.apply(lambda reg: geo_classificator(reg['keyword']), axis=1)
df_keywords.head()

,keyword,shows,content,region
0,вк,64292779,[вк],undefind
1,одноклассники,63810309,[одноклассники],undefind
2,порно,41747114,[порно],undefind
3,ютуб,39995567,[ютуб],undefind
4,вконтакте,21014195,[вконтакте],undefind


In [12]:
df_keywords.groupby('region').count().head()

,keyword,shows,content
region,,,
undefind,99253,99253,99253
Дальний Восток,94,94,94
Северо-Запад,266,266,266
Центр,387,387,387


# Задание 3 (бонусное)
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [13]:
df_movies = pd.read_csv(r'c:\Users\mkskr\OneDrive\netology\dz\dz12\ml-latest-small\movies.csv')
df_ratings = pd.read_csv(r'c:\Users\mkskr\OneDrive\netology\dz\dz12\ml-latest-small\ratings.csv')
df_movies = df_movies.merge(df_ratings, on=['movieId'] , how='left')
df_movies = df_movies.groupby(['title', 'genres']).mean().reset_index()[['title', 'genres', 'rating']].sort_values('title', ascending=False).reset_index(drop=True)
df_movies['title2']= df_movies['title'].replace(to_replace ='\)', value ='', regex =True)
df_movies.loc[:, 'content'] = df_movies['title2'].str.split('\(')
df_movies.drop(['title2'], axis='columns', inplace=True)
df_movies.head()

,title,genres,rating,content
0,İtirazım Var (2014),Action|Crime|Drama,3.500000,"[İtirazım Var , 2014]"
1,À nous la liberté (Freedom for Us) (1931),Comedy|Musical,4.500000,"[À nous la liberté , Freedom for Us , 1931]"
2,¡Three Amigos! (1986),Comedy|Western,3.258065,"[¡Three Amigos! , 1986]"
3,xXx: State of the Union (2005),Action|Crime|Thriller,1.000000,"[xXx: State of the Union , 2005]"
4,xXx (2002),Action|Crime|Thriller,2.478261,"[xXx , 2002]"


In [14]:
years = []
for i in range(1950,2011):
    years.append(str(i))
print(years)

['1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']


In [15]:
def production_year(row):
    temp_years = ''
    for i in years:
        if i in row['content']:
            temp_years = i
        else:
            continue
    if temp_years == '':
        return '1900'
    else:
        return temp_years

In [16]:
df_movies['year'] = df_movies.apply(production_year, axis=1)
df_movies.head()

,title,genres,rating,content,year
0,İtirazım Var (2014),Action|Crime|Drama,3.500000,"[İtirazım Var , 2014]",1900
1,À nous la liberté (Freedom for Us) (1931),Comedy|Musical,4.500000,"[À nous la liberté , Freedom for Us , 1931]",1900
2,¡Three Amigos! (1986),Comedy|Western,3.258065,"[¡Three Amigos! , 1986]",1986
3,xXx: State of the Union (2005),Action|Crime|Thriller,1.000000,"[xXx: State of the Union , 2005]",2005
4,xXx (2002),Action|Crime|Thriller,2.478261,"[xXx , 2002]",2002


In [17]:
df_movies = df_movies.groupby(['year']).mean().reset_index()[['year', 'rating']].sort_values('rating', ascending=False).reset_index(drop=True)
df_movies.head()

,year,rating
0,1951,3.906653
1,1952,3.802318
2,1962,3.772962
3,1953,3.738878
4,1957,3.693322
